In [0]:
import tensorflow as tf
import numpy as np
import uuid


In [0]:
n = 20  
identity = np.identity(n, dtype = np.float32)
original_states = []
flipped_states = []
for i in range(n):
    original_states.append(identity[i])
    flipped_states.append(identity[-(i+1)])




In [3]:
print(np.argmax(original_states, axis =1))
print(np.argmax(flipped_states, axis =1))


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[19 18 17 16 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1  0]


In [4]:
# tf.get_default_graph()

input_shape = [None,20]



e1_in = tf.placeholder(shape = input_shape, dtype = "float", name = "env1")
e2_in = tf.placeholder(shape = input_shape, dtype = "float", name = "env2")

e1_mid = tf.layers.dense(e1_in, 80, activation = tf.nn.relu, name = "mid1")
e2_mid = tf.layers.dense(e2_in, 80, activation = tf.nn.relu, name = "mid2")

e1_target = tf.layers.dense(e1_mid, 20, name = "target1")
e2_target = tf.layers.dense(e2_mid, 20, name = "target2")


tmp1 = tf.math.l2_normalize((e1_mid - e2_mid)**2)
tmp2 = tf.math.l2_normalize((e1_in - e1_target)**2)
tmp3 = tf.math.l2_normalize((e2_in - e2_target)**2)

mid_diff = tf.reduce_mean(tmp1)
e1_recons_cost = tf.reduce_mean(tmp2)
e2_recons_cost = tf.reduce_mean(tmp3)


cost = tf.reduce_max([e1_recons_cost , e2_recons_cost ,mid_diff])
# cost = (e1_recons_cost + e2_recons_cost + mid_diff)
optim = tf.train.RMSPropOptimizer(0.01).minimize(cost)
init = tf.global_variables_initializer()





Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [6]:
# with tf.Session() as sess:

sess = tf.Session()

sess.run(init)

for step in range(1000):
    _, cst, e1_tar, e2_tar,e1mid, e2mid = sess.run([optim,cost, e1_target, e2_target, e1_mid, e2_mid], feed_dict = {e1_in: original_states, e2_in: flipped_states})
    
    if step%100 ==0:
        print(" ")
        print(cst)
        print("mid-diff", sess.run(mid_diff, feed_dict = {e1_in: original_states, e2_in: flipped_states}) )
        print("e1_recons_cost", sess.run(e1_recons_cost, feed_dict = {e1_in: original_states, e2_in: flipped_states}) )
        print("e2_recons_cost", sess.run(e2_recons_cost, feed_dict = {e1_in: original_states, e2_in: flipped_states}) )
        print("///////////////")
        print(np.argmax(e1_tar, axis = 1))
        print(np.argmax(e2_tar, axis = 1))




 
0.015236526
mid-diff 0.015236202
e1_recons_cost 0.014287352
e2_recons_cost 0.013948354
///////////////
[16 12 17 16 14 15  3 12 12 12 11 16 15 12 12 12 16  8 16  8]
[19 19 19  2  1 18  1 13 19 19  4 13  3 19  1  3 19 19  9 10]
 
0.013998342
mid-diff 0.0139221195
e1_recons_cost 0.0139734605
e2_recons_cost 0.01396786
///////////////
[16 12 17 16 14 15  3  9 12 12 11 16 15 12 17 18 16  8 16  8]
[19 19 19  2  1 13  1 13 19 19 10 13  3 19  9  3 19 19  9 10]
 
0.003930039
mid-diff 0.0036067488
e1_recons_cost 0.003675689
e2_recons_cost 0.0038775962
///////////////
[ 0  1  2  3  4  5  6 16  8  9 10 11 12 13 14 15 16 17 18 19]
[19 18 17 16 16 14 13 12 11 10  9  8  7  6  5  4  3  2  1  0]
 
0.0025110454
mid-diff 0.0024544373
e1_recons_cost 0.0025071083
e2_recons_cost 0.0025088408
///////////////
[ 0  1  2  3  4  5  6 15  8  9 10 11 12 13 14 15 16 17 18 19]
[19 18 17 16  0 14 13 12 11 10  9  8  7  6  5  4  3  2  1  0]
 
0.0025041315
mid-diff 0.002266106
e1_recons_cost 0.0025013469
e2_recons_cos

In [0]:
mid_weights_2 = tf.get_default_graph().get_tensor_by_name('mid2/kernel:0')
mid_bias_2 = tf.get_default_graph().get_tensor_by_name('mid2/bias:0')


mid_weights_1 = tf.get_default_graph().get_tensor_by_name('mid1/kernel:0')
mid_bias_1 = tf.get_default_graph().get_tensor_by_name('mid1/bias:0')


In [8]:
print(mid_weights_2.shape)
print(mid_bias_2.shape)

print(mid_weights_2.shape)
print(mid_bias_2.shape)

(20, 80)
(80,)
(20, 80)
(80,)


In [0]:
np_weights2 = sess.run(mid_weights_2)
np_bias2 = sess.run(mid_bias_2)

np_weights1 = sess.run(mid_weights_1)
np_bias1 = sess.run(mid_bias_1)


In [0]:
print(np_weights.shape)
print(np_bias.shape)
print(np_bias)

In [0]:
import pickle as pkl

In [0]:
all_wts1 = {"W": np_weights1, "b": np_bias1}
all_wts2 = {"W": np_weights2, "b": np_bias2}

In [0]:
with open("wts_file1", "wb") as f1:
  pkl.dump(all_wts1, f1)
  
with open("wts_file2", "wb") as f2:
  pkl.dump(all_wts2, f2)
  

In [10]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['e1_input',
 'e2_input',
 'e1_middle/kernel/Initializer/random_uniform/shape',
 'e1_middle/kernel/Initializer/random_uniform/min',
 'e1_middle/kernel/Initializer/random_uniform/max',
 'e1_middle/kernel/Initializer/random_uniform/RandomUniform',
 'e1_middle/kernel/Initializer/random_uniform/sub',
 'e1_middle/kernel/Initializer/random_uniform/mul',
 'e1_middle/kernel/Initializer/random_uniform',
 'e1_middle/kernel',
 'e1_middle/kernel/Assign',
 'e1_middle/kernel/read',
 'e1_middle/bias/Initializer/zeros',
 'e1_middle/bias',
 'e1_middle/bias/Assign',
 'e1_middle/bias/read',
 'e1_middle/MatMul',
 'e1_middle/BiasAdd',
 'e1_middle/Relu',
 'e2_middle/kernel/Initializer/random_uniform/shape',
 'e2_middle/kernel/Initializer/random_uniform/min',
 'e2_middle/kernel/Initializer/random_uniform/max',
 'e2_middle/kernel/Initializer/random_uniform/RandomUniform',
 'e2_middle/kernel/Initializer/random_uniform/sub',
 'e2_middle/kernel/Initializer/random_uniform/mul',
 'e2_middle/kernel/Initializer/rand

# Case with different size of environments


In [0]:
import tensorflow as tf
import numpy as np
import pickle as pkl

In [0]:
env1_states = np.zeros(shape = [64,8,8])
env2_states = np.random.normal(0.1, 0.1, [64,8,10])

env1_states = env1_states.astype(float)
env2_states = env2_states.astype(float)

In [0]:
ind = np.array([(i, i//8, i%8) for i in range(64)])
a, b, c = ind[:, 0], ind[:, 1], ind[:, 2]

In [0]:
env1_states[a,b,c] = 1

In [0]:
env2_states[:,0:8,0:8] = env1_states

In [6]:
print(env1_states[0,3,4])
print(env2_states[0,3,5])

0.0
0.0


In [7]:
# input_shape = [None, None]

e1_in = tf.placeholder(shape = [None, 64], dtype = "float", name = "e1_input")
e2_in = tf.placeholder(shape = [None, 80], dtype = "float", name = "e2_input")


e1_mid = tf.layers.dense(e1_in, 120, activation = tf.nn.relu, name = "e1_middle")
e2_mid = tf.layers.dense(e2_in, 120, activation = tf.nn.relu, name = "e2_middle")

target_size = e1_in.shape[1]
e1_target = tf.layers.dense(e1_mid, target_size, name = "e1_target")
# e2_target = tf.layers.dense(e2_mid, flatten_shape2)


tmp1 = tf.math.l2_normalize((e1_mid - e2_mid)**2)
# tmp2 = tf.math.l2_normalize((e1_in - e1_target)**2)
# tmp3 = tf.math.l2_normalize((e2_in - e2_target)**2)


mid_diff = 300*tf.reduce_mean(tmp1)

e1_recons_cost = tf.losses.softmax_cross_entropy(e1_in, e1_target)
# e1_recons_cost = tf.reduce_mean(tmp2)
# e2_recons_cost = tf.reduce_mean(tmp3)

cost = tf.reduce_max([e1_recons_cost, mid_diff])
# cost = tf.reduce_max([e1_recons_cost , e2_recons_cost ,mid_diff])

# cost = (e1_recons_cost + mid_diff)
optim = tf.train.AdamOptimizer(0.001).minimize(cost)
init = tf.global_variables_initializer()



Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [0]:
# eg = np.asarray([0,1,0,0,0, 1, 0, 0,0,0], dtype = np.float64)
# val = tf.placeholder(shape = 10, dtype = "float")
# lw = tf.math.l2_normalize(val)
# lw1 = tf.reduce_mean(lw)

# print(" value is ", sess.run(lw1, feed_dict= {val:eg}))

In [9]:
nepochs = 150
batch_size = 25

N = env1_states.shape[0]
nsteps = N/batch_size
nsteps = int(nsteps)

sess = tf.Session()
sess.run(init)

env1_states_flat = env1_states.reshape([N,-1])
env2_states_flat = env2_states.reshape([N,-1])

for epoch in range(nepochs):

  for step in range(nsteps):
    e1batch = env1_states_flat[step*batch_size:(step+1)*batch_size, :]
    e2batch = env2_states_flat[step*batch_size:(step+1)*batch_size, :]
    
      
    _, cst, e1_tar, e1mid, e2mid = sess.run([optim,cost, e1_target, e1_mid, e2_mid], feed_dict = {e1_in: e1batch, e2_in: e2batch})
    
    print("Epoch ",epoch)
    print(cst)
    print("mid-diff", sess.run(mid_diff, feed_dict = {e1_in: e1batch, e2_in: e2batch}) )
    print("e1_recons_cost", sess.run(e1_recons_cost, feed_dict = {e1_in: e1batch, e2_in: e2batch}) )
    print(np.argmax(e1batch[1]))
    print(np.argmax(e1_tar[1]))
    print(e1batch[1])
    print(e1_tar[1])
    print("////////////////")
    print(e1mid[0][0:10])
    print(e2mid[0][0:10])

    

Epoch  0
4.172457
mid-diff 3.125806
e1_recons_cost 4.155869
1
58
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-0.16312808  0.01470991  0.12109733 -0.00499263  0.0567904  -0.04905249
 -0.11343575  0.11106057  0.01124799 -0.11498614 -0.0981127  -0.01150651
  0.05693442 -0.08662089 -0.08123855  0.178835   -0.31198832 -0.05947994
  0.03100029  0.09204081 -0.07623724 -0.13796997 -0.115114    0.04152374
  0.17788193  0.10885388 -0.00642693  0.06318579 -0.12004773  0.06748599
 -0.1748407  -0.00147386 -0.02150948  0.10940529 -0.13608897  0.04638956
  0.07031558 -0.0412763  -0.08132628 -0.05807031 -0.01724996 -0.16684562
  0.16276596 -0.07999402  0.03954154 -0.17632283  0.03345873 -0.22051436
  0.00645819  0.06066678 -0.02096008  0.18040393 -0.07553203 -0.00983188
  0.1095574  -0.08056351  0.00866383  0.01908325  0.31184396  0.02909204
  0.16520

In [0]:
mid_weights_2 = tf.get_default_graph().get_tensor_by_name('e2_middle/kernel:0')
mid_bias_2 = tf.get_default_graph().get_tensor_by_name('e2_middle/bias:0')


mid_weights_1 = tf.get_default_graph().get_tensor_by_name('e1_middle/kernel:0')
mid_bias_1 = tf.get_default_graph().get_tensor_by_name('e1_middle/bias:0')


In [13]:
print(mid_weights_2.shape)
print(mid_bias_2.shape)

print(mid_weights_1.shape)
print(mid_bias_1.shape)

(80, 120)
(120,)
(64, 120)
(120,)


In [0]:
np_weights2 = sess.run(mid_weights_2)
np_bias2 = sess.run(mid_bias_2)

np_weights1 = sess.run(mid_weights_1)
np_bias1 = sess.run(mid_bias_1)

In [0]:
all_wts1 = {"W": np_weights1, "b": np_bias1}
all_wts2 = {"W": np_weights2, "b": np_bias2}

In [0]:
with open("wts_file1", "wb") as f1:
  pkl.dump(all_wts1, f1)
  
with open("wts_file2", "wb") as f2:
  pkl.dump(all_wts2, f2)

In [13]:
arr = [[[1,2],[3,4]],[[5,6],[7,8]],[[9,10],[11,12]],[[13,14],[15,16]]]
arr = np.asarray(arr, dtype = np.float32)
print(arr.shape)

(4, 2, 2)


In [0]:
t = arr.reshape([4,-1])

In [12]:
print(t)

[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]
 [13. 14. 15. 16.]]
